In [1]:
!pip install pyDOE
!pip install cma

  Created wheel for pyDOE: filename=pyDOE-0.3.8-cp36-none-any.whl size=18178 sha256=56ffabcc372a10dfac35e789947d39158cb70567667e4758e0b949aef145e197
  Stored in directory: /root/.cache/pip/wheels/7c/c8/58/a6493bd415e8ba5735082b5e0c096d7c1f2933077a8ce34544
Successfully built pyDOE
     |████████████████████████████████| 245kB 2.8MB/s 


In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
import bbobbenchmarks as bn
import sklearn
from sklearn.preprocessing import PolynomialFeatures

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F3 '''
def F3(X):
    f = bn.F3()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data


''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))


''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 1.03 ,random_state=0 , l1_ratio=1.0, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)


''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 23.95 , C = 992.98 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_3_4000Samples.csv"
train = pd.read_csv(path).iloc[:,1:]
test = pd.read_csv('test_3_800Samples.csv').iloc[:,1:]
true = np.array(test['Y'])

## Surrogate Models

In [5]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


## CMA-ES

In [6]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,201):
        Cols.append(Columns[j]+'_X'+str(i))
const = [ [-5] * 200, [5] * 200 ]

opt = cma.CMAOptions()
opt.set("bounds", const)
opt.set ("seed" , 0)
opt.set ("maxfevals" , 200000)

n_obs , dim =  30, 200
G = DOE(n_obs, dim)
G  = [ linearscaletransform(G[idx] , range_out=(-5,5)) for idx in range(dim) ]
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([10,600])
for i in range(0,10):
  print ('Run : '+ str(i))
  min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  X_Values [i,:] = list(min_kri)+list(min_svr)+list(min_eln)

Run : 0
(9_w,19)-aCMA-ES (mu_w=5.6,w_1=29%) in dimension 200 (seed=977816, Tue Apr 14 16:36:54 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     19 1.831064235333808e+04 1.0e+00 2.43e+00  2e+00  2e+00 0:00.1
    2     38 1.701743221892358e+04 1.0e+00 2.38e+00  2e+00  2e+00 0:00.2


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [21244.67694452] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [20254.22295171] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


    3     57 1.636989233648975e+04 1.0e+00 2.33e+00  2e+00  2e+00 0:00.4
   28    532 4.231004768340696e+03 1.1e+00 1.94e+00  2e+00  2e+00 0:03.5
   61   1159 1.584159251411897e+03 1.1e+00 1.63e+00  2e+00  2e+00 0:07.5
  100   1900 -6.848621016164150e+03 1.2e+00 1.21e+00  1e+00  1e+00 0:12.1
  150   2850 -1.526926539675378e+04 1.2e+00 7.31e-01  7e-01  7e-01 0:18.2
  200   3800 -2.283791765701103e+04 1.2e+00 4.25e-01  4e-01  4e-01 0:24.2
  266   5054 -2.692601150527676e+04 1.2e+00 2.38e-01  2e-01  2e-01 0:32.3
  300   5700 -2.768028389126569e+04 1.2e+00 1.79e-01  2e-01  2e-01 0:36.3
  382   7258 -2.833942934489485e+04 1.2e+00 8.43e-02  8e-02  8e-02 0:46.4
  400   7600 -2.840221386747999e+04 1.2e+00 7.24e-02  7e-02  7e-02 0:48.6
  499   9481 -2.851239743138070e+04 1.2e+00 2.48e-02  2e-02  2e-02 1:00.6
  500   9500 -2.851272666401302e+04 1.2e+00 2.45e-02  2e-02  2e-02 1:00.7
  600  11400 -2.852686568217391e+04 1.2e+00 8.89e-03  8e-03  9e-03 1:12.8
  700  13300 -2.852875572177140e+04 1.3e+

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [22322.25886811] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=22322.258868, sigma=2.44e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=22322.258868, sigma=2.44e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  '

termination on tolfun=1e-11 (Tue Apr 14 16:58:13 2020)
final/bestever f-value = 2.232226e+04 2.232226e+04
incumbent solution: [ 4.53118389 -0.12662079  1.46897223 -2.5553036   2.07331735 -2.24159369
 -3.66220251  2.08355786 ...]
std deviations: [2.43746098 2.43742716 2.43698058 2.43727026 2.43702141 2.43729652
 2.43762395 2.43689146 ...]
(9_w,19)-aCMA-ES (mu_w=5.6,w_1=29%) in dimension 200 (seed=945029, Tue Apr 14 16:58:13 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     19 1.754798746682983e+04 1.0e+00 2.44e+00  2e+00  2e+00 0:00.1
    2     38 1.656253956731037e+04 1.0e+00 2.38e+00  2e+00  2e+00 0:00.2


...

    3     57 1.773091629150218e+04 1.0e+00 2.34e+00  2e+00  2e+00 0:00.4
   28    532 6.140527626598017e+03 1.1e+00 1.99e+00  2e+00  2e+00 0:03.4
   61   1159 3.040138605314525e+03 1.1e+00 1.82e+00  2e+00  2e+00 0:07.5
  100   1900 1.236854440423529e+03 1.2e+00 1.52e+00  2e+00  2e+00 0:12.3
  149   2831 -6.561928914430984e+03 1.2e+00 1.10e+00  1e+00  1e+00 0:18.3
  200   3800 -1.842310964173326e+04 1.3e+00 6.74e-01  7e-01  7e-01 0:24.6
  266   5054 -2.543105256900834e+04 1.3e+00 3.44e-01  3e-01  3e-01 0:32.7
  300   5700 -2.651652723280302e+04 1.3e+00 2.53e-01  2e-01  3e-01 0:36.8
  382   7258 -2.819064762083774e+04 1.3e+00 1.16e-01  1e-01  1e-01 0:46.8
  400   7600 -2.822839684630683e+04 1.3e+00 9.73e-02  9e-02  1e-01 0:49.1
  499   9481 -2.848372848911715e+04 1.3e+00 3.77e-02  4e-02  4e-02 1:01.2
  500   9500 -2.848617779721726e+04 1.3e+00 3.73e-02  4e-02  4e-02 1:01.3
  600  11400 -2.852293273696671e+04 1.3e+00 1.48e-02  1e-02  1e-02 1:13.6
  700  13300 -2.852833616844225e+04 1.3e+0

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [22322.25886811] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=22322.258868, sigma=2.44e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=22322.258868, sigma=2.44e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  '

termination on tolfun=1e-11 (Tue Apr 14 22:49:25 2020)
final/bestever f-value = 2.232226e+04 2.232226e+04
incumbent solution: [-0.28950228  1.71026371  4.64205208  1.21951005 -3.45765869  4.67038574
 -3.36564422 -2.79991585 ...]
std deviations: [2.43614444 2.43631173 2.43632109 2.43651395 2.43615613 2.43642626
 2.43658247 2.43638044 ...]
(9_w,19)-aCMA-ES (mu_w=5.6,w_1=29%) in dimension 200 (seed=908405, Tue Apr 14 22:49:26 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     19 2.094446611675668e+04 1.0e+00 2.44e+00  2e+00  2e+00 0:00.1
    2     38 1.685079448562766e+04 1.0e+00 2.39e+00  2e+00  2e+00 0:00.2


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [21610.84095865] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [23967.50533285] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


    3     57 1.754636321416195e+04 1.0e+00 2.34e+00  2e+00  2e+00 0:00.3
   31    589 7.040546260530209e+03 1.1e+00 1.96e+00  2e+00  2e+00 0:03.4
   68   1292 4.675196762304018e+03 1.1e+00 1.74e+00  2e+00  2e+00 0:07.4
  100   1900 2.963260429421811e+03 1.2e+00 1.52e+00  2e+00  2e+00 0:10.9
  156   2964 8.380205177119860e+02 1.2e+00 1.08e+00  1e+00  1e+00 0:17.0
  200   3800 -2.736732697770072e+03 1.3e+00 7.85e-01  8e-01  8e-01 0:21.7
  274   5206 -4.917229673732756e+03 1.3e+00 5.28e-01  5e-01  5e-01 0:29.8
  300   5700 -6.088599463141465e+03 1.3e+00 4.35e-01  4e-01  4e-01 0:32.6
  392   7448 -7.815062018630822e+03 1.3e+00 2.59e-01  3e-01  3e-01 0:42.7
  400   7600 -7.776122822363148e+03 1.3e+00 2.49e-01  2e-01  3e-01 0:43.6
  500   9500 -8.646845952440179e+03 1.4e+00 1.50e-01  1e-01  2e-01 0:54.5
  600  11400 -9.001148093274182e+03 1.5e+00 1.07e-01  1e-01  1e-01 1:05.5
  700  13300 -9.224561865064567e+03 1.6e+00 8.11e-02  8e-02  9e-02 1:16.4
  800  15200 -9.305703360960637e+03 1.7e+00

In [7]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('Results\\F3_X_Values_first.csv')
Krig_Fun = np.zeros(10)
SVM_Fun = np.zeros(10)
ELN_Fun = np.zeros(10)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F3(X_Values.iloc[i,:200])
    SVM_Fun [i] = F3(X_Values.iloc[i,200:400])
    ELN_Fun [i] = F3(X_Values.iloc[i,400:600])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([4292.73427555]), count=array([1])) 0.00013771541698889784
SVM
ModeResult(mode=array([19854.37364465]), count=array([1])) 2756.4881630065333
ELN
ModeResult(mode=array([10244.06765671]), count=array([1])) 273.87708960413846
